# Binomial Model

## DataSet
> The data is from market bond price model

In [1]:
from zero import zeros, forward_rate
import pandas as pd

bond_data = [
    [100, 0.50, 0, 99.1338],
    [100, 1.00, 0, 97.8925],
    [100, 1.50, 0, 96.1531]
]

# calculate zero coupon bond's yield
_yield = list(map(lambda x: round(x, 4), zeros(bond_data)))
print("Yield: " + str(_yield))
# calculate forward rate
_forward = list(map(lambda x: round(x, 4), forward_rate(bond_data, 0.5)))
print("Forward Rate: " + str(_forward))
print()

market_data = pd.DataFrame(
    {
        "Maturity": [0.5, 1.0, 1.5],
        "Price": [99.1338, 97.8925, 96.1531],
        "Yield": _yield,
        "Forward Rate": _forward
    }
)

print(market_data)

Yield: [0.0174, 0.0213, 0.0262]
Forward Rate: [0.0174, 0.0252, 0.0359]

   Maturity    Price   Yield  Forward Rate
0       0.5  99.1338  0.0174        0.0174
1       1.0  97.8925  0.0213        0.0252
2       1.5  96.1531  0.0262        0.0359


## Interest Rate Model
> Using binomial model and get one-step and two-step


In [2]:
import binomial as bi

irm1 = bi.model(
    [
        0.0174,
        [0.0339, 0.0095]
    ]
)
print("1step: ")
print(str(irm1.tree))
print()
irm2 = bi.model(
    [
        0.0174,
        [0.0339, 0.0095],
        [0.0500, 0.0256, 0.0256, 0.0011]
    ]
)
print("2step: ")
print(str(irm2.tree))

1step: 
[[[0], 0.0174], [[0, 0], 0.0339], [[0, 1], 0.0095]]

2step: 
[[[0], 0.0174], [[0, 0], 0.0339], [[0, 1], 0.0095], [[0, 0, 0], 0.05], [[0, 0, 1], 0.0256], [[0, 1, 0], 0.0256], [[0, 1, 1], 0.0011]]


> Before get Bond Price Model, you should calc RNP from one-step irm

In [3]:
from price import rnp

# Get market price at maturity 1 year, 1.5 year
mp0 = market_data.Price[0]
mp1 = market_data.Price[1]
mp2 = market_data.Price[2]

RNP = rnp(100, irm1, mp1, 0.5)
print("RNP: " + str(RNP))

RNP: 0.6448615964742744


## Bond Price Model
> Draw up bond price model from interest rate model

In [4]:
from price import bond_model, round_model

bpm1 = round_model(bond_model(100, irm1, [RNP], 0.5), 4)
print("Bond Price(maturity = 1): ")
print(str(bpm1.tree))
print()

bpm2 = round_model(bond_model(100, irm2, [RNP, RNP], 0.5), 4)
print("Bond Price(maturity = 1.5): ")
print(str(bpm2.tree))

Bond Price(maturity = 1): 
[[[0], 97.8925], [[0, 0], 98.3193], [[0, 1], 99.5261], [[0, 0, 0], 100], [[0, 0, 1], 100], [[0, 1, 0], 100], [[0, 1, 1], 100]]

Bond Price(maturity = 1.5): 
[[[0], 96.3137], [[0, 0], 96.3098], [[0, 1], 98.6904], [[0, 0, 0], 97.531], [[0, 0, 1], 98.7282], [[0, 1, 0], 98.7282], [[0, 1, 1], 99.945], [[0, 1, 0, 0], 100], [[0, 1, 0, 1], 100], [[0, 1, 0, 0], 100], [[0, 1, 0, 1], 100], [[0, 1, 1, 0], 100], [[0, 1, 1, 1], 100], [[0, 1, 1, 0], 100], [[0, 1, 1, 1], 100]]


## Replicating Portfolio
> Make portfolio that price is same as bond option

### Derivative Price

In [27]:
from price import model_price

# Derivative price
rk = 0.02
r10 = irm1.value(1, 0)
r11 = irm1.value(1, 1)
p10 = bpm1.value(1, 0)
p11 = bpm1.value(1, 1)
payoff = lambda x, y: 100*max(x-y, 0)
drp_up = round(payoff(rk, r10), 2)
drp_dn = round(payoff(rk, r11), 2)

derivative_price_data = pd.DataFrame(
    {
        "Scenario 6m": ["up", "dn"],
        "Interest Rate": [r10, r11],
        "6m Bond": [100, 100],
        "1yr Bond": [p10, p11],
        "Derivative Price": [drp_up, drp_dn]
    }
)

print(derivative_price_data)

  Scenario 6m  Interest Rate  6m Bond  1yr Bond  Derivative Price
0          up         0.0339      100   98.3193              0.00
1          dn         0.0095      100   99.5261              1.05


### Portfolio Value

In [28]:
# Portfolio value
_return = lambda x, y: round(x/y, 3)

portfolio_value_data = pd.DataFrame(
    {
        "Scenario in 6m": ["up", "dn"],
        "6m Bond return": [_return(100, mp0), _return(100, mp0)],
        "1yr Bond return": [_return(p10, mp1), _return(p11, mp1)],
        "Portfolio Value": [f"{_return(100, mp0)}X+{_return(p10, mp1)}Y", 
                           f"{_return(100, mp0)}X+{_return(p11, mp1)}Y"]
    }
)

print(portfolio_value_data)

  Scenario in 6m  6m Bond return  1yr Bond return Portfolio Value
0             up           1.009            1.004   1.009X+1.004Y
1             dn           1.009            1.017   1.009X+1.017Y


> solve X and Y with matrix algebra

In [31]:
from price import simultaneous_equation
import numpy as np

_coef = np.matrix([
    [_return(100, mp0), _return(p10, mp1)],
    [_return(100, mp0), _return(p11, mp1)]
])

# Make (1, 2) to (2, 1)
_from = derivative_price_data["Derivative Price"]
_to = list(map(lambda x: [x], _from))
_target = np.matrix(_to)

solution = simultaneous_equation(_coef, _target)
print("solution is:")
print(solution)
print()

portfolio_value = float(solution[0][0]+solution[1][0])
print("Portfolio Value = "+str(portfolio_value))

[[1.009 1.004]
 [1.009 1.017]]
solution is:
[[-80.36898681]
 [ 80.76923077]]

Portfolio Value = 0.4002439582221484
